In [ ]:
%%capture
!pip install -U langchain langchain_experimental openai

In [ ]:
# set environment variables
import os
os.environ["OPENAI_API_KEY"] = ""

# Imports

In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.pydantic_v1 import BaseModel
from datetime import datetime
from typing import List, Optional
from langchain_experimental.tabular_synthetic_data.base import SyntheticDataGenerator
from langchain_experimental.tabular_synthetic_data.openai import create_openai_data_generator, OPENAI_TEMPLATE
from langchain_experimental.tabular_synthetic_data.prompts import SYNTHETIC_FEW_SHOT_SUFFIX, SYNTHETIC_FEW_SHOT_PREFIX

# Schema for generating Grouping




In [ ]:
class IntrusionSet(BaseModel):
    type: str
    spec_version: str
    id: str
    created: str
    modified: str
    name: str
    description: Optional[str] = None
    aliases: Optional[str] = None
    first_seen: Optional[str] = None
    last_seen: Optional[str] = None
    goals: Optional[str] = None
    resource_level: Optional[str] = None
    primary_motivation: Optional[str] = None
    secondary_motivation: Optional[str] = None



# Sample Data as example

In [ ]:
examples = [
    {"example": """Type: intrusion_set, spec_version: 2.1, id: intrusion-set--4e78f46f-a023-4e5f-bc24-71b3ca22ec29, created: 2016-04-06T20:03:48.000Z, modified: 2016-04-06T20:03:48.000Z, name: Bobcat Breakin, description: Incidents usually feature a shared TTP of a bobcat being released within the building containing network access, scaring users to leave their computers without locking them first. Still determining where the threat actors are getting the bobcats., aliases: zookeeper, goals: acquistion-theft"""},
    {"example": """Type: intrusion_set, spec_version: 2.1, id:intrusion-set--da1065ce-972c-4605-8755-9cd1074e3b5a, created: 2015-05-15T09:12:16.432Z, modified: 2015-05-15T09:12:16.432Z, name: APT1, description: APT1 is a single organization of operators that has conducted a cyber espionage campaign against a broad range of victims since at least 2006, first_seen: 2006-06-01T18:13:15.684Z, resource_level: government, primary motivation: organizational-gain, aliases: Comment Crew, Comment Group, shady rat"""},
    {"example": """Type: intrusion_set, spec_version: 2.1, id: intrusion-set--ed69450a-f067-4b51-9ba2-c4616b9a6713, created: 2016-08-08T15:50:10.983Z, modified: 2016-08-08T15:50:10.983Z, name: APT BPP, description: An advanced persistent threat that seeks to disrupt Branistan's election with multiple attacks., aliases: bran-teaser, first_seen: 2016-01-08T12:50:40.123Z, goals: Influence the Branistan election, disrupt the BPP, resource_level: government, primary_motivation: ideology, secondary_motivation: dominance"""},
]

# Prompt Template for GPT-4

In [ ]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

# Data Generator

In [ ]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=IntrusionSet,
    llm=ChatOpenAI(temperature=1,model='gpt-4-turbo-preview'),
    prompt=prompt_template,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Parameters

In [ ]:
synthetic_results = synthetic_data_generator.generate(
    subject="Intrusion Set ",
    extra="Choose a unique and unconventional description, aliases, first seen, last seen, goals, resource level, primary motivation, and secondary motivation for each Infrastructure. Avoid common or typical names.",
    runs=1,
)

In [ ]:
len(synthetic_results)

1

# Display Data

In [ ]:
synthetic_results

[IntrusionSet(type='intrusion_set', spec_version='2.1', id='intrusion-set--5f2c3df2-7b82-4f60-9a47-1268b1f8f65e', created='2022-02-24T11:45:00.000Z', modified='2022-02-24T11:45:00.000Z', name='Midnight Mischief', description='This group is known for conducting cyber operations strictly during local nighttime hours, exploiting the reduced operational presence. Their signature move involves the digital equivalent of graffiti, leaving whimsical images and messages on affected systems. Origin and motivations are still under investigation.', aliases='Night Owls, Moondust Vandals', first_seen='2021-11-30T02:15:45.000Z', last_seen='2023-03-15T09:35:00.000Z', goals='Generate public attention, Humiliate target organizations', resource_level='cluster', primary_motivation='personal-satisfaction', secondary_motivation='competition')]

# Display as a DataFrame

In [ ]:
import pandas as pd

# Create a list of dictionaries from the objects
synthetic_data = []
for item in synthetic_results:
    synthetic_data.append({
        'type': item.type,
        'name': item.name,
        'description': item.description,
        'aliases': item.aliases,
        'first_seen': item.first_seen,
        'last_seen': item.last_seen,
        'goals': item.goals,
        'resource_level': item.resource_level,
        'primary_motivation': item.primary_motivation,
        'secondary_motivation': item.secondary_motivation
        })

# Create a Pandas DataFrame from the list of dictionaries
synthetic_df = pd.DataFrame(synthetic_data)

# Display the DataFrame
print(type(synthetic_df))
synthetic_df

<class 'pandas.core.frame.DataFrame'>


,type,name,description,aliases,first_seen,last_seen,goals,resource_level,primary_motivation,secondary_motivation
0,intrusion_set,Midnight Mischief,This group is known for conducting cyber opera...,"Night Owls, Moondust Vandals",2021-11-30T02:15:45.000Z,2023-03-15T09:35:00.000Z,"Generate public attention, Humiliate target or...",cluster,personal-satisfaction,competition


In [ ]:
# Save the DataFrame to a CSV file
synthetic_df.to_csv('intrusion_set.csv', index=False)  # index=False prevents adding an extra index column
print("intrusion set data saved to 'intrusion_set.csv'")

intrusion set data saved to 'intrusion_set.csv'
